In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from torchvision.datasets.mnist import FashionMNIST
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import models
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
import math

print(torch.__version__)
device = torch.device('cuda:1')

In [ ]:
class calTech(Dataset):
    def __init__(self, name, transform=None):
        hf = h5py.File(name, 'r')
        self.input_images = np.array(hf.get('features'))
        self.input_labels = np.array(hf.get('labels')).astype(np.long)
        self.transform = transform
        hf.close()

    def __len__(self):
        return (self.input_images.shape[0])

    def __getitem__(self, idx):
        images = self.input_images[idx]
        labels = self.input_labels[idx]
        if self.transform is not None:
            images = self.transform(images)
        return images, labels

In [ ]:
INPUT_CHANNEL = 3
BATCH_SIZE = 1
normalize = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

classes = [str(i) for i in range(102)]

data_path = os.path.dirname(os.getcwd()) + "/data/calTech/"
Train_data = calTech(data_path + "train.h5", transform=normalize)
Train_dataloader = DataLoader(dataset=Train_data, batch_size = 1, shuffle=True)

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


class My_Model(nn.Module):
    def __init__(self, input_channel=1, num_class=21):
        super(My_Model, self).__init__()
        self.model_ft = models.vgg16(pretrained=True)
        set_parameter_requires_grad(self.model_ft, False) 
        #change FC
        self.model_ft.classifier[6] = nn.Linear(4096, num_class)
        
    def forward(self, x):
        # Perform the usual forward pass
        x_class = self.model_ft(x)
            
        return F.softmax(x_class, dim=1)

In [ ]:
_model = My_Model(num_class=102)
_model.to(device)

In [ ]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
inv_normalize = transforms.Normalize([-0.485/0.229, -0.456/0.224, -0.406/0.225], [1/0.229, 1/0.224, 1/0.225])

def tensor_to_img(t):
    """Convert normalized tensor in Cuda to cv2 image"""
    unnormalized = inv_normalize(t)
    npimg = np.transpose(unnormalized.cpu().numpy(), (1, 2, 0))
    return npimg

def img_to_cuda_tensor(img):
    tr_img = np.transpose(img, (2, 0, 1))
    t = torch.from_numpy(tr_img)
    t = normalize(t.float())
    return t.to(device)
    

def imshow(img, title):
    """Custom function to display the image using matplotlib"""    
    npimg = tensor_to_img(img)
    #plot the numpy image
    plt.figure(figsize = (4, 4))
    plt.axis("off")
    plt.imshow(npimg)
    plt.title(title)
    plt.show()
    

def imshow_multi(list_img, title = None, n_cols = 5):
    n_rows = math.ceil((len(list_img))/n_cols)
    fig, axes = plt.subplots(n_rows,n_cols, figsize=(4*n_cols,4*n_rows))
    for i,ax in enumerate(axes.flat):
        ax.grid(False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if i>=len(list_img):
            continue
        if(title != None):
            ax.set_title("layer " + str(title[i]))

        ax.imshow(list_img[i])
        
    plt.tight_layout()
    

In [ ]:
class SaveFeatures():
    """hook function at forward step"""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output
    def close(self):
        self.hook.remove()

class InvertedRepresentation():
    def __init__(self, model):
        self.model = model
        self.model.eval()

    def alpha_norm(self, input_matrix, alpha):
        """
            Normal norm
        """
        alpha_norm = ((input_matrix.view(-1))**alpha).sum()
        return alpha_norm

    def total_variation_norm(self, input_matrix, beta):
        """
            Total variation norm
        """
        to_check = input_matrix[:, :-1, :-1]  # Trimmed: right - bottom
        one_bottom = input_matrix[:, 1:, :-1]  # Trimmed: top - right
        one_right = input_matrix[:, :-1, 1:]  # Trimmed: top - right
        total_variation = (((to_check - one_bottom)**2 +
                            (to_check - one_right)**2)**(beta/2)).sum()
        return total_variation

    def euclidian_loss(self, org_matrix, target_matrix):
        """
            Normalized Euclidian loss ||fi(x) - fi(x_0)||_2^2& / ||fi(x_0)||_2^2
        """
        distance_matrix = target_matrix - org_matrix
        euclidian_distance = self.alpha_norm(distance_matrix, 2)
        normalized_euclidian_distance = euclidian_distance / self.alpha_norm(org_matrix, 2)
        return normalized_euclidian_distance

    def get_activations(self, x, layer_id):
        """
            get feature maps
        """
        
        activations = SaveFeatures(self.model.model_ft.features[layer_id])
        output = self.model(x)
        feature_map = activations.features[0]
        return feature_map

    def generate_inverted_image(self, input_image, alpha = (6, 1e-7), beta = (2, 1e-8) ,target_layer=3):
        # Generate a random image which we will optimize
        input_image = input_image.to(device).float()
        img_size = input_image.size(2)
        
        rand_img = torch.randn(1, 3, img_size, img_size).to(device)
        opt_img = Variable(1e-1 * rand_img, requires_grad=True)
        
        optimizer = torch.optim.SGD([opt_img], lr=1e4, momentum=0.9)
        
        # Get the output with the real input image
        input_image_layer_output = self.get_activations(input_image, target_layer)

        # Alpha regularization parametrs
        alpha_reg_alpha, alpha_reg_lambda = alpha
       
        # Total variation regularization parameters
        tv_reg_beta, tv_reg_lambda = beta
       
        recreated_im = []
        for i in (range(200)):
            optimizer.zero_grad()
            # Get the output with the random input image
            output = self.get_activations(opt_img, target_layer)
            # Calculate euclidian loss
            euc_loss = 1e-1 * self.euclidian_loss(input_image_layer_output.detach(), output)
            # Calculate alpha regularization
            reg_alpha = alpha_reg_lambda * self.alpha_norm(opt_img, alpha_reg_alpha)
            # Calculate total variation regularization
            reg_total_variation = tv_reg_lambda * self.total_variation_norm(opt_img, tv_reg_beta)
            # Sum all to optimize (all channels)
            loss = euc_loss + reg_alpha + reg_total_variation
            
            # Step
            loss.backward()
            optimizer.step()
            # save image every 5 iterations
            if i % 5 == 0:
                recreated_im.append(tensor_to_img(opt_img.data[0]))
            # Reduce learning rate every 40 iterations
            if i % 40 == 0:
                for param_group in optimizer.param_groups:
                    param_group['lr'] *= 1/10
        #print('Loss:', loss.item())
        
        return recreated_im
    
def reconstruct_single_layer(inv, input_image, alpha = (6, 1e-6), beta = (2, 1e-6), target_layers=[3]):
    list_reconstruct = []
    for layer_index in target_layers:
        reconstructs = inv.generate_inverted_image(input_image, alpha, beta, target_layer = layer_index)
        list_reconstruct.append(reconstructs[-1])
    imshow_multi(list_reconstruct, title=target_layers)
    
    return list_reconstruct

In [ ]:
inv = InvertedRepresentation(_model)

input_image, label = next(iter(Train_dataloader))
imshow(input_image[0], classes[label])
re = reconstruct_single_layer(inv, input_image, target_layers = range(0, 30))

In [ ]:
input_image, label = next(iter(Train_dataloader))
imshow(input_image[0], classes[label])
re = reconstruct_single_layer(inv, input_image, target_layers = range(0, 30))

In [ ]:
input_image, label = next(iter(Train_dataloader))
imshow(input_image[0], classes[label])
re = reconstruct_single_layer(inv, input_image, target_layers = range(0, 30))

In [ ]:
input_image, label = next(iter(Train_dataloader))
imshow(input_image[0], classes[label])
re = reconstruct_single_layer(inv, input_image, target_layers = range(0, 30))

In [ ]:
input_image, label = next(iter(Train_dataloader))
imshow(input_image[0], classes[label])
re = reconstruct_single_layer(inv, input_image, target_layers = range(0, 30))